## IEMS 308 - Dillard's SKU Analysis

The Dillard's SKU dataset can be [downloaded here](https://www.dropbox.com/s/5o0v3uoakkqe8u6/Dillards%20POS.7z?dl=0).

The final project report can be [found here](www.google.com).


In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import random
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from tabulate import tabulate
import matplotlib

### Data Preprocessing

Because this dataset is so large, we need to in some "clever way, select a subset of data". 

To do this: 
1. Load the `strinfo` dataset and take a random sample of `stores` to pick. 
    1. This drastically reduces the datasize required to parse.
    2. Store 9404 is in Oklahoma City, OK - this is chosen store to develop plantograms for.
   
2. Load just the barebones of the `transact` table, dropping out the unnecessary columns and keeping only `sku`,`store`, `register`,`trannum`, `seq`, `saledate`, `stype`.
    1. One good method is to use `chunksize` because otherwise Python will not let the file be loaded into memory. It seems 1,000,000 is a good chunksize. 
    2. As each chunk is collected, only retain the relevant store transaction data.
    3. The chunks can be concatenated into a fully-functional dataframe.
3. Now that the barebones have been made into a functional dataframe, drop all columns except `sku`, `trannum` and `stype`.
    1. It's important to ensure these are integer values and all whitespace is removed.

In [ ]:
str_cols = ['store', 'city', 'state', 'zip']
strinfo = pd.read_csv('dillard/strinfo.csv', sep=',',header=None, usecols=range(0, 4), names=str_cols)

In [ ]:
# stores = random.sample(strinfo.store.tolist(), 1)
# strinfo[strinfo['state'] == 'TX'].head(10)

stores=[9404] #focus on Oklahoma City, OK
strinfo[strinfo['store'].isin(stores)]

In [ ]:
#transcation data
transaction_col = ['sku', 'store', 'register', 'trannum', 'seq', 'saledate', 'stype']
transactions_iter = pd.read_csv('dillard/trnsact.csv', sep=',', header=None,
                            chunksize=1000000, usecols=range(0, 7), names=transaction_col)

In [ ]:
temp_list = []
for trans_iter in transactions_iter:
    cleaned = trans_iter[(trans_iter.store.isin(stores))]
    temp_list.append(cleaned)
transactions_compiled = pd.concat(temp_list) # this will be the entire (subsetted) dataframe of trnsact.csv

1. Keep only `stype=P` since only purchases contribute to revenue.
2. Removing duplicate records.
3. Intra-transaction SKUs are represented as just 1 instance instead of many.
4. SKUs that were represented less than 5 instances across transaction table were removed due to lack of prevalence.
5. All remaining SKUs were one-hot encoded so that they appeared as binary indicators per transaction row to signify if a particular SKU was purchased in that particular transaction.

In [ ]:
# transactions_compiled.groupby('store').describe()
transactions_compiled.head()

In [ ]:
print(transactions_compiled.shape)
#stores = [2409, 1904, 6700, 3202, 8702]

In [ ]:
#Remove all columns except sku, store trannum, stype
transactions_compiled = transactions_compiled[['sku','store','trannum','stype']].copy()
transactions_compiled.head(5)

In [ ]:
transactions_compiled.groupby('store').describe()

In [ ]:
#Obtain subset of dataframe that contains only purchases
transactions_compiled = transactions_compiled[transactions_compiled['stype'].str.contains('P')]

In [ ]:
#now that we only have purchases, drop the col
transactions_compiled = transactions_compiled.drop('stype',axis=1)

In [ ]:
#make sure that the data is the right type
transactions_compiled.dtypes

In [ ]:
sku_counts_s = transactions_compiled['sku'].value_counts() #this is a pandas series
type(sku_counts_s)

In [ ]:
sku_counts_df = sku_counts_s.to_frame() #convert series to dataframe
sku_counts_df = sku_counts_df.reset_index()
sku_counts_df.columns = ['sku','count']
print(type(sku_counts_df))

In [ ]:
transaction_counts = pd.merge(transactions_compiled,sku_counts_df, on=['sku']) #merge content

In [ ]:
transactions_noduplicates = transaction_counts.drop_duplicates() #drop any duplicates

In [ ]:
# Create subset of tran_noduplicates where all counts are 5 or higher
transactions_clean = transactions_noduplicates[transactions_noduplicates['count'] > 4]

In [ ]:
transactions_clean.head(5)

# #One hot encode the transactions so that each row is a unique transaction and each column denotes 
# #the presence of an SKU (1) or the lack of an SKU (0) for the given transaction
# df_biloxi_encoded = pd.get_dummies(df_biloxi_before_encoding,columns=['sku'])
# del(df_biloxi_encoded['count'])
# df_biloxi_reencoded = df_biloxi_encoded.groupby('trannum').sum()

# #Run the apriori algorithm
# frequent_itemsets = apriori(df_biloxi_reencoded,min_support=0.103, use_colnames=True)
# rules = association_rules(frequent_itemsets,metric="lift")

My jupyter kernel keeps dying at this step, so I'll save the dataframe to csv just to be sure we can keep making progress.

In [ ]:
transactions_clean.to_csv('transactions_clean.csv',index=False)

In [ ]:
transactions_clean=pd.read_csv('transactions_clean.csv',sep=',')

In [ ]:
transactions_dummy = pd.get_dummies(transactions_clean,columns=['sku'])

In [ ]:
transactions_dummy.head(5)

In [ ]:
del(transactions_dummy['count'])
del(transactions_dummy['store'])

In [ ]:
transactions_dummy.shape

In [ ]:
t_twice_encoded = transactions_dummy.groupby('trannum').sum()

In [ ]:
t_twice_encoded.shape

In [ ]:
# transactions_dummy_df = pd.concat([transactions_clean,transactions_dummy],axis=1)

In [ ]:
# transactions_dummy_df = transactions_dummy_df.drop('sku',axis=1)

In [ ]:
# transactions_dummy_df.shape

In [ ]:
# transactions_dummy_df.head()

In [ ]:
# del(transactions_dummy_df['count'])
# del(transactions_dummy_df['store'])

In [ ]:
# transactions_dummy_df.head()

In [ ]:
# t_twice_encoded = transactions_dummy_df.groupby('trannum').sum()

In [ ]:
t_twice_encoded

### Applying Associative Rule Analysis

1. Run Algorithm.
2. Compute `support`, `lift`, `confidence` metrics. 
    1. Only keep if `support` > 0.25 AND `confidence` > 0.5 AND `lift` > 3
        1. OR
    2. IF`support` > 0.6 
        2. OR
    3. `confidence` > 0.75 
        3. OR 
    4. `lift` > 4

In [ ]:
#Run the apriori algorithm
frequent_itemsets = apriori(t_twice_encoded,min_support=0.103, use_colnames=True)

In [ ]:
rules = association_rules(frequent_itemsets,metric="lift")

In [ ]:
rules[:10]

In [ ]:
#Eliminate rules with antecedents having over 2 items
rules_remainder = rules[rules['antecedents'].map(len)<=2]

In [ ]:
useful_rules = rules_remainder[((rules_remainder['support']>.25) & (rules_remainder['confidence']>.5) & (rules_remainder['lift']>3)) | \
             ((rules_remainder['support']>.6) | (rules_remainder['confidence']>.75) | (rules_remainder['lift']>4))]


In [ ]:
useful_rules = useful_rules.reset_index()

In [ ]:
#Determine the total number of unique antecedent SKUs in useful_rules
antecedents = []
for i in range(0,np.shape(useful_rules)[0]):
    antecedents.append(list(useful_rules['antecedents'].iloc[i])[0])
    if len(useful_rules['antecedents'].iloc[i])==2:
        antecedents.append(list(useful_rules['antecedents'].iloc[i])[1])
antecedents = np.asarray(antecedents)

In [ ]:
#Add most frequently bought SKUs to the list
frequent_counts = transactions_clean.sort_values(by='count',ascending=False).drop_duplicates(subset='sku')

In [ ]:
antecedent_integers = []
for num in antecedents:
    antecedent_integers.append(int(num[4:]))

In [ ]:
cnt = 0    
total_skus = np.unique(antecedent_integers)
while len(total_skus)<100:
    if frequent_counts['sku'].iloc[cnt] not in total_skus:
        total_skus = np.append(total_skus,frequent_counts['sku'].iloc[cnt])
    cnt+=1

In [ ]:
#Output final SKUs to move
print(total_skus)